In [2]:
import numpy as np
import librosa
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [6]:
y, sr = librosa.load('wav폴더\\test1.wav', sr=22050) # 1번 wav파일 load

mfcc = librosa.feature.mfcc(y=y, sr=sr)  # load한 1번 wav파일을 mfcc 적용. shape 는 (20x216) 20은 나눠준 프레임수 216은 각 프레임의 특징값 수

x_std = StandardScaler().fit_transform(mfcc)  # mfcc적용한 값들을 rescaling 해서 모두 같은 범위 안에 있도록 해준다.

print(y,sr)
print(mfcc.shape)


[ 0.00995745  0.0182266  -0.00475176 ... -0.2698473  -0.26954573
 -0.27955517] 22050
(20, 216)


In [7]:
features = x_std.T  # 216개의 특징값에 대한 covariance matrix를 만들기 위해선 216개의 특징값을 row로 바꿔야한다.
# 그래서 x_std에 .T를 붙여 transpose를 해줌.

In [8]:
features.shape # 그럼 (20x216) 이던게 (216x20) 으로 바뀜

(216, 20)

In [9]:
covariance_matrix = np.cov(features) # 216개의 특징값에 대한 covariance matrix를 생성
covariance_matrix.shape  # 그럼 covariance matrix는 (216x216) 이 된다.

(216, 216)

In [10]:
eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)  # np.linalg.eig 를 사용해서 covariance_matrix의 고유값과 고유벡터를 구함.

In [11]:
print(eig_vals)

[ 2.26430138e+02+0.00000000e+00j  1.85850844e-01+0.00000000e+00j
  1.61910985e-01+0.00000000e+00j  9.78936726e-02+0.00000000e+00j
  7.78678272e-02+0.00000000e+00j  6.62864664e-02+0.00000000e+00j
  5.67783566e-02+0.00000000e+00j  5.13292988e-02+0.00000000e+00j
  4.29834732e-02+0.00000000e+00j  3.86072222e-02+0.00000000e+00j
  3.51611134e-02+0.00000000e+00j  2.75719523e-02+0.00000000e+00j
  2.25427692e-02+0.00000000e+00j  1.98497560e-02+0.00000000e+00j
  1.58320523e-02+0.00000000e+00j  1.38362720e-02+0.00000000e+00j
  1.16236578e-02+0.00000000e+00j  8.48210451e-03+0.00000000e+00j
  3.87544639e-03+0.00000000e+00j  1.55855557e-14+0.00000000e+00j
 -1.35912802e-14+0.00000000e+00j  3.81034416e-15+1.26718328e-14j
  3.81034416e-15-1.26718328e-14j  1.18531591e-14+0.00000000e+00j
  8.73910933e-15+6.94919028e-15j  8.73910933e-15-6.94919028e-15j
 -9.42618188e-15+5.27146900e-15j -9.42618188e-15-5.27146900e-15j
 -4.81473458e-15+7.88183498e-15j -4.81473458e-15-7.88183498e-15j
  8.90406555e-15+7.707987

In [66]:
eig_vals[0]/sum(eig_vals)  # 고유값 중의 최대값의 비중을 확인하면 99.60% 라는 거의 대부분의 정보를 이 한개의 값이 담고있다는것을 알 수 있다.

(0.9960864836732894+0j)

In [12]:
projected_X = x_std.dot(eig_vecs.T[0]) # dot 연산은 행렬을 곱해주는 연산이다. 즉 A.dot(B) 이렇게 있으면 AxB 의 행렬을 구해준다
# 때문에 x_std는 (20x216) 이었고 eig_vecs.T[0] 은 (216x1) 이니까 곱해주면 20 array가 나온다.
projected_X.shape # 20 array 형태가 나온다.

(20,)

In [13]:
y = np.asmatrix(projected_X) #  20 어레이 형태를 (1x20) 행렬로 만들어준다
a = y  # a 에는 이제 계속 정상 데이터들로만 담을거임
b = np.array([0]) # 0 이라는 값을 담은 어레이를 하나 만든다.(이 0 은 나중이 ocsvm을 돌릴때 label로 사용될거임)
b = b.reshape(-1,1) # 0 이라는 값을 컬럼값으로 바꿔주고
a = np.hstack((a,b)) # hstack 을 사용하여 0 값을 컬럼에 합쳐주어 (1x21) 을 만든다.
a.shape

(1, 21)

In [15]:
for i in range(2,2872):   # 불량을 제외한 나머지 모든 정상데이터들에 대해 위의 행위를 적용시켜 concate 시켜서 최종적으로 (2862x21) 을 만든다.
    if i==1119 or i==2495 or i==2442 or i==1905 or i==2596 or i==2393 or i==2211 or i==2603 or i==2605:
        continue
    y, sr = librosa.load('wav폴더\\test{}.wav'.format(i), sr = 22050)

    mfcc = librosa.feature.mfcc(y=y, sr=sr)

    x_std = StandardScaler().fit_transform(mfcc)
    
    features = x_std.T
    
    covariance_matrix = np.cov(features)
    
    eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
    
    projected_X = x_std.dot(eig_vecs.T[0])
    
    y = np.asmatrix(projected_X)
    b = y
    c = np.array([0])
    c = c.reshape(-1,1)
    b = np.hstack((b,c))
    a = np.concatenate((a,b),axis=0)
    
a.shape

(2862, 21)

In [70]:
np.save('train_mfcc20',a)

In [71]:
y, sr = librosa.load('wav폴더\\test1119.wav'.format(i), sr = 22050)  # 똑같이 이번엔 불량 데이터들을 만들어준다.

mfcc = librosa.feature.mfcc(y=y, sr=sr)

x_std = StandardScaler().fit_transform(mfcc)
    
features = x_std.T
    
covariance_matrix = np.cov(features)
    
eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
    
projected_X = x_std.dot(eig_vecs.T[0])
    
y = np.asmatrix(projected_X)
a = y
b = np.array([1])  # 불량 데이터의 label은 1 로 한다.
b = b.reshape(-1,1)
a = np.hstack((a,b))

a.shape

(1, 21)

In [72]:
for i in range(2,2872):  # 불량 데이터들만 concate
    if i==2495 or i==2442 or i==1905 or i==2596 or i==2393 or i==2211 or i==2603 or i==2605:
        y, sr = librosa.load('wav폴더\\test{}.wav'.format(i), sr = 22050)

        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        x_std = StandardScaler().fit_transform(mfcc)
    
        features = x_std.T
    
        covariance_matrix = np.cov(features)
    
        eig_vals, eig_vecs = np.linalg.eig(covariance_matrix)
    
        projected_X = x_std.dot(eig_vecs.T[0])
    
        y = np.asmatrix(projected_X)
        b = y
        c = np.array([1])
        c = c.reshape(-1,1)
        b = np.hstack((b,c))
        a = np.concatenate((a,b),axis=0)
    
a.shape

(9, 21)

In [73]:
np.save('test_mfcc20',a)